# **Lab 1: matrix factorization**
**Jesper Lidbaum**

# **Abstract**

In this report, three algorithms are implemented from pseudo-code provided in chapter 5 in [Johan Hoffman, Methods in Computational Science](https://my.siam.org/Store/Product/viewproduct/?ProductId=39300058). Lastly, two of the algorithms are combined to make a solver for systems of linear equations.

#**About the code**

A short statement on who is the author of the file, and if the code is 

---

distributed under a certain license. 

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2023 Jesper Lidbaum (jlidbaum@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [ ]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**
The algorithms in this lab report are all based on pseudo code that can be found in [Johan Hoffman, Methods in Computational Science](https://my.siam.org/Store/Product/viewproduct/?ProductId=39300058). The statements about the complexity and implementation details are also all derived from there. The last task is however not entirely from the course book. But it can be derived from it.


## Sparse matrix-vector multiplication
A problem with multiplying large matrices with vectors is that an algorithm for dense matrices has a complexity of $\mathcal O(n²)$. But this can be improved when dealing with a sparse matrix, which means a matrix that has a lot of zeros. We use a special data structure when storing sparse matrices that contain 3 arrays. One with the non-zero elements, one for their column, and an array containing pointers to where the rows begin and end in the first array. This implementation is useful because it is unnecessary to store many elements that are zero. 

## QR factorization
QR factorization is a method of factorizing a matrix real, quadratic, and invertible matrix into an orthogonal and upper triangular matrix. This report is done with the modified Gram-Schmidt method. The code is based on algorithm 5.3 from the course book. This factorization is useful because an orthogonal matrix is easily inverted by transposing it, and an upper triangular matrix can be inverted by substitution. This then provides an inverse for A.

## Direct solving
As mentioned before we can generate an inverse for a quadratic and real matrix $A$ by applying the $QR$ factorization and then inverting the result. This can be used to solve equations on the form $Ax=b$. The eqation can be rewritten as $QRx=b <=> x=R^{-1}Q^{-1}b$. And since Q is orthogonal this can further be rewritten as $x=R^{-1}Q^Tb$. R is upper triangular which can be calculated with backward substitution. This is implemented in the report with algorithm 5.2 from the course book.

# **Method**

In this section, the code used to solve the different tasks are presented.

##Algortihms

### Sparse matrix-vector multiplication
sparse_matrix_vector_product is a function that takes a vector and a sparse matrix. The sparse matrix data structure is defined with 3 arrays. First the non-zero values, then the index that the values are on in their column, and lastly an array of pointers to which value each row begins on. 



In [ ]:
#Implementation of algorithm 5.9 from the course book. Chapter 5, page 101
def sparse_matrix_vector_product(x, val, col_idx, row_ptr):
    b = np.zeros(len(row_ptr) - 1)
    for i in range(len(row_ptr) - 1):
        for j in range(row_ptr[i], row_ptr[i + 1]):
            b[i] = b[i] + val[j] * x[col_idx[j]]

    return b

# QR factorization
modified_gram_schmidt_iteration is based on algorithm 5.3 from the course book. The algorithm takes a matrix $A$ and factorizes it into the orthogonal matrix $Q$ and the upper triangular matrix $R$.

In [ ]:
#Implementation of algorithm 5.3 from the course book. Chapter 5, page 89
def modified_gram_schmidt_iteration(A):
    Q = np.zeros(A.shape)
    R = np.zeros(A.shape)
    v = np.zeros(A.shape[0])
    for j in range(A.shape[1]):
        v[:] = A[:,j].copy()
        for i in range(j):
            R[i,j] = np.dot(Q[:,i], v[:])
            v[:] = v[:] - R[i,j] * Q[:,i]
        R[j,j] = np.linalg.norm(v)
        Q[:,j] = np.multiply(v[:], 1.0/R[j,j])
    return Q, R

# Direct solving
The direct solver uses the $QR$ decomposition gained from modified_gram_schmidt_iteration. It then solves the equation $Ax=b$ by calculating the value of $x=R^{-1}Q^Tb$ in two steps. First, the transpose of $Q$ is multiplied by b. We let $y=Q^Tb$. Then $Rx=y$ is solved using backward subsitution. 

In [ ]:
#Implementation of algorithm 5.2 from the course book. Chapter 5, page 87
def backward_substitution(R, b):
    n = R.shape[0]
    x = np.zeros(n)
    x[n-1] = b[n-1] / R[n-1, n-1]
    for i in range(n-2, -1, -1):
        sum = 0
        for j in range(i+1, n):
            sum = sum + R[i,j] * x[j]
        x[i] = (b[i] - sum) / R[i,i]
    return x

def direct_solver(A, b):
    Q, R = modified_gram_schmidt_iteration(A)
    Qt = np.transpose(Q)
    y = Qt.dot(b)
    x = backward_substitution(R, y)
    return x

## Tests
Here we define some test methods. The tests are from the lab instructions. 

### Sparse matrix-vector multiplication
We test the equation $\left(\begin{array}{ccc} 
1 & 2 & 3\\
4 & 5 & 6\\
7 & 8 & 9\\
\end{array}\right)
\left(\begin{array}{c} 
1\\
2\\
3
\end{array}\right)$ with our implementation against numpy's implementation of dense multiplication. The residual norm between the results is returned to indicate the difference.

In [ ]:
def test_sparse_matrix_vector_product():
    A = np.array([[1,2,3], [4,5,6], [7,8,9]])
    val = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
    col_idx = np.array([0, 1, 2, 0, 1, 2, 0, 1, 2])
    row_ptr = np.array([0, 3, 6, 9])
    x = np.array([1, 2, 3])
    b = sparse_matrix_vector_product(x, val, col_idx, row_ptr)
    return (np.linalg.norm(b - A.dot(x)))

### QR factorization
Similarly, we check the result of the factorization. $Q^TQ$ should be equal to the identity matrix. And $QR$ should be equal to $A$. We test the result with the Forbenius norm which is the default for [numpy norms](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html). 

In [ ]:
def test_modified_gram_schmidt_iteration():
    A = np.array([[2, -1], [-1, 2]])
    Q, R = modified_gram_schmidt_iteration(A)
    return (np.linalg.norm(Q.dot(R) - A)), (np.linalg.norm(Q.dot(np.transpose(Q)) - np.identity(2)))

### Direct solving
We test the direct solving with the residual norm between a fabricated result. This with numpy's solver and our own.

In [ ]:
def test_direct_solver():
    A = np.array([[2, -1], [-1, 2]])
    b = np.array([100, 10])
    x = direct_solver(A, b)
    y = np.linalg.solve(A, b)
    return (np.linalg.norm(A.dot(x) - b)), (np.linalg.norm(x - y))

# **Results**

We present the results from the different tests in this section.

### Sparse matrix-vector multiplication


In [ ]:
print(test_sparse_matrix_vector_product())

0.0


### QR factorization

In [ ]:
print(test_modified_gram_schmidt_iteration())

(0.0, 3.1836122848239643e-16)


### Test direct solving

In [ ]:
print(test_direct_solver())

(1.4210854715202004e-14, 7.105427357601002e-15)


# **Discussion**

The conclusion is that the implemented algorithms seems to work for the tests. All the norms are either very small or 0. This is however no proof of correctness. But these algorithms are implementations of psuedo code that is well researched and a correct implementation should work. I was suprised at how well the direct solver worked, it had a simple implementation.

The psuedo code was also written in a way that was easily translated to python. However the differences in starting index is something that you need to be aware of.